In [40]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from src.data.data_fetcher import get_all_features, get_raw_data
from src.features.feature_engineering import prepare_data
from src.features.preprocess_data import fetch_preprocessed_data, get_preprocessed_test_data

%pip install tensorflow
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

Note: you may need to restart the kernel to use updated packages.


In [41]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

In [42]:
X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
X_test_estimated_a_processed, X_test_estimated_b_processed, X_test_estimated_c_processed = get_preprocessed_test_data()

c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:326: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample("1H").mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:326: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample("1H").mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:326: FutureWarning: The default value of numeric_o

In [43]:
(X_train_obs_combined.shape, X_val_obs_combined.shape, y_train_obs_combined.shape, y_val_obs_combined.shape, \
 X_train_est_combined.shape, X_val_est_combined.shape, y_train_est_combined.shape, y_val_est_combined.shape)

((2343, 45), (586, 45), (2343,), (586,), (440, 45), (110, 45), (440,), (110,))

In [46]:

# Scale the feature data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train_obs_combined)
y_train_scaled = scaler_Y.fit_transform(y_train_obs_combined.values.reshape(-1, 1))
X_val_scaled = scaler_X.transform(X_train_obs_combined)
y_val_scaled = scaler_Y.transform(y_train_obs_combined.values.reshape(-1, 1))
WINDOW_SIZE = 1
# Reshape input to be [samples, time steps, features]
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], WINDOW_SIZE, X_train_scaled.shape[1]))
X_val_reshaped = X_val_scaled.reshape((X_val_scaled.shape[0], WINDOW_SIZE, X_val_scaled.shape[1]))

# Construct the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val_scaled), verbose=2, shuffle=False)
model.summary()

Epoch 1/50
74/74 - 2s - loss: 0.0184 - val_loss: 0.0130 - 2s/epoch - 30ms/step
Epoch 2/50
74/74 - 0s - loss: 0.0114 - val_loss: 0.0115 - 209ms/epoch - 3ms/step
Epoch 3/50
74/74 - 0s - loss: 0.0107 - val_loss: 0.0105 - 205ms/epoch - 3ms/step
Epoch 4/50
74/74 - 0s - loss: 0.0090 - val_loss: 0.0081 - 230ms/epoch - 3ms/step
Epoch 5/50
74/74 - 0s - loss: 0.0084 - val_loss: 0.0078 - 202ms/epoch - 3ms/step
Epoch 6/50
74/74 - 0s - loss: 0.0087 - val_loss: 0.0095 - 202ms/epoch - 3ms/step
Epoch 7/50
74/74 - 0s - loss: 0.0087 - val_loss: 0.0078 - 201ms/epoch - 3ms/step
Epoch 8/50
74/74 - 0s - loss: 0.0088 - val_loss: 0.0080 - 200ms/epoch - 3ms/step
Epoch 9/50
74/74 - 0s - loss: 0.0086 - val_loss: 0.0091 - 223ms/epoch - 3ms/step
Epoch 10/50
74/74 - 0s - loss: 0.0085 - val_loss: 0.0078 - 198ms/epoch - 3ms/step
Epoch 11/50
74/74 - 0s - loss: 0.0085 - val_loss: 0.0078 - 201ms/epoch - 3ms/step
Epoch 12/50
74/74 - 0s - loss: 0.0080 - val_loss: 0.0081 - 202ms/epoch - 3ms/step
Epoch 13/50
74/74 - 0s - lo

In [ ]:

# Make predictions on validation data
y_val_pred_scaled = model.predict(X_val_reshaped)

# Transform predictions back to original scale
y_val_pred = scaler_Y.inverse_transform(y_val_pred_scaled).flatten()
# Calculate Mean Absolute Error on validation data
# mae_val = mean_absolute_error(y_val_obs_combined, y_val_pred)
# mae_val

74/74 [==============================] - 1s 1ms/step


In [ ]:
# Make predictions using the trained Linear Regression model on the processed test estimated data
from src.features.preprocess_data import get_final_prediction

print(X_test_estimated_a_processed.shape)

print(X_val_reshaped.shape)
scaler_X = MinMaxScaler(feature_range=(0, 1))

X_test_estimated_a_processed = scaler_X.fit_transform(X_test_estimated_a_processed)
X_test_estimated_b_processed = scaler_X.fit_transform(X_test_estimated_b_processed)
X_test_estimated_c_processed = scaler_X.fit_transform(X_test_estimated_c_processed)

X_test_estimated_a_reshaped = X_test_estimated_a_processed.reshape((X_test_estimated_a_processed.shape[0], WINDOW_SIZE, X_test_estimated_a_processed.shape[1]))
X_test_estimated_b_reshaped = X_test_estimated_b_processed.reshape((X_test_estimated_b_processed.shape[0], WINDOW_SIZE, X_test_estimated_b_processed.shape[1]))
X_test_estimated_c_reshaped = X_test_estimated_c_processed.reshape((X_test_estimated_c_processed.shape[0], WINDOW_SIZE, X_test_estimated_c_processed.shape[1]))

y_pred_test_est_a = model.predict(X_test_estimated_a_reshaped)
y_pred_test_est_b = model.predict(X_test_estimated_b_reshaped)
y_pred_test_est_c = model.predict(X_test_estimated_c_reshaped)
# Combine the predictions from the three locations into a single dataframe
y_pred_test_est_a = scaler_Y.inverse_transform(y_pred_test_est_a).flatten()
y_pred_test_est_b = scaler_Y.inverse_transform(y_pred_test_est_b).flatten()
y_pred_test_est_c = scaler_Y.inverse_transform(y_pred_test_est_c).flatten()

y_pred_test_est_combined = get_final_prediction(y_pred_test_est_a, y_pred_test_est_b, y_pred_test_est_c)
y_pred_test_est_combined.head(20)


(2880, 45)
(2343, 1, 45)
90/90 [==============================] - 0s 1ms/step


,id,time,location,prediction
0,0,2023-05-01 00:00:00,A,-16.772728
1,1,2023-05-01 01:00:00,A,-19.422329
2,2,2023-05-01 02:00:00,A,-16.257036
3,3,2023-05-01 03:00:00,A,-17.789270
4,4,2023-05-01 04:00:00,A,-0.951706
5,5,2023-05-01 05:00:00,A,21.030996
6,6,2023-05-01 06:00:00,A,43.099777
7,7,2023-05-01 07:00:00,A,57.981331
8,8,2023-05-01 08:00:00,A,78.886810
9,9,2023-05-01 09:00:00,A,105.028503


## Step : Save model

In [ ]:

from src.models.saving import save_predictions


save_predictions(y_pred_test_est_combined, 'lstm_only_obs_no_feature_engineering')

   id  prediction
0   0         0.0
1   1         0.0
2   2         0.0
3   3         0.0
4   4         0.0
